# Get the cmd path

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

# Install

In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install -q supervision roboflow
!conda install -c huggingface huggingface_hub -y

In [ ]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt
!ls -lh {HOME}/weights

In [ ]:
!mkdir -p {HOME}/data
!wget -P {HOME}/data -q https://media.roboflow.com/notebooks/examples/dog.jpeg
!ls -lh {HOME}/data

# Inference

In [ ]:
import cv2
from ultralytics import YOLOv10

## pretrained model
# model = YOLOv10(f'{HOME}/weights/yolov10n.pt')
# results = model(source=f'{HOME}/data/dog.jpeg', conf=0.25)

## custom model
model = YOLOv10(f'{HOME}/runs/detect/train13/weights/best.pt')
image = cv2.imread(f'/path/to/testMenu.jpg')
results = model(image)[0]

In [ ]:
## Get results (boxes, labels)
results.boxes
# results[0].boxes.xyxy
# results[0].boxes.cls

## Draw out the annotated image

In [ ]:
import cv2
import supervision as sv
from ultralytics import YOLOv10

model = YOLOv10(f'{HOME}/runs/detect/train13/weights/best.pt')
image = cv2.imread(f'/path/to/testMenu.jpg')
results = model(image)[0]
detections = sv.Detections.from_ultralytics(results)

bounding_box_annotator = sv.BoundingBoxAnnotator(thickness=1)
label_annotator = sv.LabelAnnotator(
    text_position=sv.Position.CENTER,
    text_scale=0.2,
    text_padding=0,
)

annotated_image = bounding_box_annotator.annotate(
    scene=image, detections=detections)

for detection_idx in range(len(detections)):
    x1, y1, x2, y2 = detections.xyxy[detection_idx].astype(int)
    color = (0, 0, 0)
    annotated_image = cv2.rectangle(
        img=annotated_image,
        pt1=(x1, y1),
        pt2=(x2, y2),
        color=color,
        thickness=-1,
    )

annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)